In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from plotnine import *
from sklearn.linear_model import LinearRegression
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose, STL

In [ ]:
os.getcwd()

In [ ]:
df=pd.read_csv('Strut_Assemblies_Python.csv')
df.head()

In [ ]:
df.dtypes

In [ ]:
df['Day'] = "1"
df['Year']=df['Year'].astype(str)
df['Month_Num']=df['Month_Num'].astype(str)
# df['Month1']=df['Month_Num']
# df['Year_Month']=pd.to_datetime(df[["Year", "Month", "Day"]])
df['Year_Month']=pd.to_datetime(df.Year+df.Month_Num+df.Day, format="%Y%m%d")
df.rename(columns={'Price':'Price'}, inplace=True)
df.rename(columns={'Units':'Qty'}, inplace=True)
df.rename(columns={'Part/Sku':'Material'}, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna(axis=0, subset=('Price', ))

In [ ]:
# df['Price'] = df['Price'].astype(float)

In [ ]:
df.head()

In [ ]:
# # Choose the top 80 SKU's
# df=df[df['SKU Tag']=='Top 80%']

In [ ]:
df.head()

In [ ]:
# #Check values in different PH2
# category = df['PH2'].value_counts()
# print("PH2 Category Percentage\n{}\n{}\n PH2 Category values count\n{}\n{}".format(
#       50*"-", (category / len(df.index))[:5] * 100, 50*"-", category[:5]))

In [ ]:
#Check values in different Life Cycle
LC = df['LC'].value_counts()
print("LC Percentage\n{}\n{}\n LC values count\n{}\n{}".format(
      50*"-", (LC / len(df.index))[:5] * 100, 50*"-", LC[:5]))

## Without Exclusion of Data

In [ ]:
#Aggregate the data
df_sku = df.groupby(['Year_Month','Material']).agg({'Price':'mean','Qty': 'mean' }).reset_index()
# df_sku=df_sku[(df_sku.Year_Month<'2022-10-01') & (df_sku.Year_Month>='2020-07-01')]
df_sku.head()

# df_sku = df.groupby(['Year_Month','Material']).agg({'Sales':'sum','Qty': 'sum' }).reset_index()
# df_sku['Price']=df_sku['Sales']/df_sku['Qty']
# df_sku.head()
# df_LC['Year_Month'].unique()

In [ ]:
# df_sku=df_sku.drop('Sales',axis=1)
# df_sku.head()

In [ ]:
# Mean, Max, Min by LCs
prices = df_sku.groupby('Material').agg({'Price' : ['mean', 'min', 'max']})
prices.columns = ['price_mean','price_min', 'price_max']
prices["price_diff"] = prices["price_max"] - prices["price_min"]
prices = prices.reset_index()
prices.sort_values(by="price_diff", ascending = False, inplace = True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
     print(prices)

In [ ]:
# #Box Plot
# pct_threshold = 99.0
# # censor below threshold
# pct95facet = np.percentile(df_sku['Price'], pct_threshold)

# box_dist = (
#     ggplot(df_sku) + 
#     geom_boxplot(
#         aes(x = 'LC',
#             y = 'Price'), colour="#1F3552", fill="#4271AE", 
#                    alpha=0.7,
#                    outlier_shape=".",
#                    outlier_colour="red"       
#     ) +
#     labs(
#         title ='China Damper by LC',
#         x = 'LC',
#         y = 'Price',
#     ) +
#     scale_y_continuous() +
#     geom_hline(yintercept = pct95facet) + # add percentile solid line
#     theme(axis_text_x = element_text(angle = 45, hjust = 1)) + 
#     theme(figure_size=(20, 12)) + 
#     theme(text=element_text(family="Tahoma", size=12)) +
#     theme(axis_text_x=element_text(colour="black", size=12)) +
#     theme(axis_text_y=element_text(colour="black", size=12))
# )
# box_dist

In [ ]:
## Data Prep using pivot
x_pivot = df_sku.pivot(index= 'Year_Month' ,columns='Material' ,values='Price')
x_values = pd.DataFrame(x_pivot.to_records())
print(x_values)

In [ ]:
percent_missing_x = x_values.isnull().sum() * 100 / len(x_values)
missing_value_df_x = pd.DataFrame({'column_name': x_values.columns,
                                 'percent_missing': percent_missing_x})
missing_value_df_x.sort_values(by=['percent_missing'])

In [ ]:
mod_missing_value_df_x=missing_value_df_x[missing_value_df_x.percent_missing<=20]
mod_missing_value_df_x

In [ ]:
mod_missing_value_df_x=mod_missing_value_df_x.reset_index(drop=True)
mod_missing_value_df_x['column_name']

In [ ]:
x_values_mod = x_values[x_values.columns[x_values.columns.isin(mod_missing_value_df_x['column_name'].astype('str'))]]
x_values_mod

In [ ]:
y_pivot = df_sku.pivot( index = 'Year_Month',columns='Material', values='Qty')
y_values = pd.DataFrame(y_pivot.to_records())
print(y_values)

In [ ]:
percent_missing_y = y_values.isnull().sum() * 100 / len(x_values)
missing_value_df_y = pd.DataFrame({'column_name': y_values.columns,
                                 'percent_missing': percent_missing_y})
missing_value_df_y.sort_values(by=['percent_missing'])

In [ ]:
mod_missing_value_df_y=missing_value_df_y[missing_value_df_y.percent_missing<=20]

In [ ]:
mod_missing_value_df_y=mod_missing_value_df_y.reset_index(drop=True)
mod_missing_value_df_y['column_name']

In [ ]:
y_values_mod = y_values[y_values.columns[y_values.columns.isin(mod_missing_value_df_y['column_name'].astype('str'))]]
y_values_mod

In [ ]:

results_values_raw = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_detrended = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_deseasonalized = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_log_raw = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_log_detrended = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_log_deseasonalized = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}


In [ ]:
for column in x_values_mod.columns[1:]:
    column_points = []
    for i in range(len(x_values_mod[column])):
        if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]):
            # column_points.append((x_values_mod['Year_Month'][i],x_values_mod[column][i], y_values_mod[column][i]))
            column_points.append((i+1,x_values_mod['Year_Month'][i],x_values_mod[column][i], y_values_mod[column][i]))
    # df = pd.DataFrame(list(column_points), columns= ['Date','x_value', 'y_value'])
    df = pd.DataFrame(list(column_points), columns= ['Date_Index','Date','x_value', 'y_value'])
    # print(df)
    # sns.regplot(x="x_value", y="y_value", data=df, fit_reg=True)
    # plt.xlabel('Price')
    # plt.ylabel('Quantity')
    # plt.title('Raw Price vs Raw Quantity')
    # plt.show()
    
    #Slope
    x_value = df['Date_Index']
    y_value = df['y_value']
    X = sm.add_constant(x_value)
    model_x = sm.OLS(y_value, X)
    result_x = model_x.fit()
    intercept, slope = result_x.params
    df['y_value_detrended'] = df['y_value'] - (df['Date_Index']*slope)
    # print(df) 
    # sns.lineplot(data = df[['y_value','y_value_detrended']])
    # plt.title('Detrended Plot')
    # plt.show()
    
    # df.reset_index(inplace=False)
    df = df.set_index('Date')
    df.sort_index(inplace=True)
    df = df.resample('MS').asfreq().fillna(0)
    res = seasonal_decompose(df.y_value, extrapolate_trend='freq')
    # df['y_value'] = res.trend
    df['y_value_deseasonalized'] = df.y_value_detrended.values - res.seasonal
    df = pd.DataFrame(df)
    # print(df)
    # sns.lineplot(data = df[['y_value','y_value_detrended','y_value_deseasonalized']])
    # plt.title('Deseasonalized Plot')
    # plt.show()
    
    # df['log_x_value'] = np.log(df['x_value'])
    # df['log_y_value'] = np.log(df['y_value'])
    # df['log_y_value_detrended'] = np.log(df['y_value_detrended'])
    # df['log_y_value_deseasonalized'] = np.log(df['y_value_deseasonalized'])
    
    df['log_x_value'] = np.where(df['x_value'] > 0, np.log(df['x_value']) , 0)
    df['log_y_value'] = np.where(df['y_value'] > 0, np.log(df['y_value']) , 0) 
    df['log_y_value_detrended'] = np.where(df['y_value_detrended'] > 0, np.log(df['y_value_detrended']) , 0)
    df['log_y_value_deseasonalized'] = np.where(df['y_value_deseasonalized'] > 0, np.log(df['y_value_deseasonalized']) , 0)
    # print(df)
    
    

#     df.reset_index(inplace=False)
#     df = df.set_index('Date')
#     df_ts=df[['y_value']]
#     # print(df_ts)
#     sns.lineplot(data = df_ts)
#     plt.title('Time Series of Raw Quantity')
#     plt.show()
#     # fig, ax = plt.subplots(figsize = (12,6))    
#     # fig = sns.lineplot(x = "Date", y='y_value',data=df_ts,ax=ax)

#     # df.reset_index(inplace=False)
#     # df = df.set_index('Date')
#     res = seasonal_decompose(df.y_value, extrapolate_trend='freq')
#     # df['y_value'] = res.trend
#     df['y_value'] = df.y_value.values - res.seasonal
#     df = pd.DataFrame(df)
#     print(df)
#     sns.lineplot(data = df['y_value'])
#     plt.title('Deseasonalized Time Series of Raw Quantity')
#     plt.show()
#     # df['y_value'] = df.y_value.values - res.trend
    
#     # res = STL(df_ts, robust = True).fit()
#     # df_ts['y_value'] = df_ts['y_value'] - res.seasonal
#     # df_ts = pd.DataFrame(df_ts)
#     # print(df_ts)
#     # sns.lineplot(data = df_ts[['y_value']])
#     # plt.title('Deseasonalized Time Series of Raw Quantity')
#     # plt.show()
    
    
# #     df = pd.DataFrame(df)
# #     print(df)
# #     sns.lineplot(data = df[['y_value']])
# #     plt.title('Deseasonalized Time Series of Raw Quantity')
# #     plt.show()

#     df=df.reset_index(drop=True)
# #     df = df[(df.x_value < df.x_value.quantile(.95)) & (df.x_value > df.x_value.quantile(.05))]
#     sns.regplot(x="x_value", y="y_value", data=df, fit_reg=True)
#     plt.xlabel('Price')
#     plt.ylabel('Quantity')
#     plt.title('Raw Price vs Deseasonalized Quantity')
#     plt.show()


    #Raw Model
    x_value = df['x_value']
    y_value = df['y_value']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_raw["name"].append(column)
    results_values_raw["price_elasticity"].append(price_elasticity)
    results_values_raw["price_mean"].append(mean_price)
    results_values_raw["quantity_mean"].append(mean_quantity)
    results_values_raw["intercept"].append(intercept)
    results_values_raw['t_score'].append(t_score)
    results_values_raw["slope"].append(slope)
    results_values_raw["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_raw["rsquared"].append(rsquared*100)
    
    #Detrended
    x_value = df['x_value']
    y_value = df['y_value_detrended']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_detrended["name"].append(column)
    results_values_detrended["price_elasticity"].append(price_elasticity)
    results_values_detrended["price_mean"].append(mean_price)
    results_values_detrended["quantity_mean"].append(mean_quantity)
    results_values_detrended["intercept"].append(intercept)
    results_values_detrended['t_score'].append(t_score)
    results_values_detrended["slope"].append(slope)
    results_values_detrended["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_detrended["rsquared"].append(rsquared*100)
    
    #Deseasonalized Model
    x_value = df['x_value']
    y_value = df['y_value_deseasonalized']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_deseasonalized["name"].append(column)
    results_values_deseasonalized["price_elasticity"].append(price_elasticity)
    results_values_deseasonalized["price_mean"].append(mean_price)
    results_values_deseasonalized["quantity_mean"].append(mean_quantity)
    results_values_deseasonalized["intercept"].append(intercept)
    results_values_deseasonalized['t_score'].append(t_score)
    results_values_deseasonalized["slope"].append(slope)
    results_values_deseasonalized["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_deseasonalized["rsquared"].append(rsquared*100)

   #Log Raw Model
    x_value = df['log_x_value']
    y_value = df['log_y_value']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log_raw["name"].append(column)
    results_values_log_raw["price_elasticity"].append(price_elasticity)
    results_values_log_raw["price_mean"].append(mean_price)
    results_values_log_raw["quantity_mean"].append(mean_quantity)
    results_values_log_raw["intercept"].append(intercept)
    results_values_log_raw['t_score'].append(t_score)
    results_values_log_raw["slope"].append(slope)
    results_values_log_raw["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log_raw["rsquared"].append(rsquared*100)
    
    #log Detrended
    x_value = df['log_x_value']
    y_value = df['log_y_value_detrended']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log_detrended["name"].append(column)
    results_values_log_detrended["price_elasticity"].append(price_elasticity)
    results_values_log_detrended["price_mean"].append(mean_price)
    results_values_log_detrended["quantity_mean"].append(mean_quantity)
    results_values_log_detrended["intercept"].append(intercept)
    results_values_log_detrended['t_score'].append(t_score)
    results_values_log_detrended["slope"].append(slope)
    results_values_log_detrended["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log_detrended["rsquared"].append(rsquared*100)
    
    #log Deseasonalized Model
    x_value = df['log_x_value']
    y_value = df['log_y_value_deseasonalized']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log_deseasonalized["name"].append(column)
    results_values_log_deseasonalized["price_elasticity"].append(price_elasticity)
    results_values_log_deseasonalized["price_mean"].append(mean_price)
    results_values_log_deseasonalized["quantity_mean"].append(mean_quantity)
    results_values_log_deseasonalized["intercept"].append(intercept)
    results_values_log_deseasonalized['t_score'].append(t_score)
    results_values_log_deseasonalized["slope"].append(slope)
    results_values_log_deseasonalized["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log_deseasonalized["rsquared"].append(rsquared*100)
    
df_raw = pd.DataFrame.from_dict(results_values_raw)
df_raw['ID'] = '1-Raw'
df_raw_detrended = pd.DataFrame.from_dict(results_values_detrended)
df_raw_detrended['ID'] = '5-Raw_Detrended'
df_raw_deseasonalized = pd.DataFrame.from_dict(results_values_deseasonalized)
df_raw_deseasonalized['ID'] = '6-results_values_deseasonalized'
df_raw_log = pd.DataFrame.from_dict(results_values_log_raw)
df_raw_log['ID'] = '4-Log_Raw'
df_raw_detrended_log = pd.DataFrame.from_dict(results_values_log_detrended)
df_raw_detrended_log['ID'] = '2-Log_Raw_Detrended'
df_raw_deseasonalized_log = pd.DataFrame.from_dict(results_values_log_deseasonalized)
df_raw_deseasonalized_log['ID'] = '3-Log_results_values_deseasonalized'
final_df = pd.concat([df_raw,df_raw_detrended,df_raw_deseasonalized,df_raw_log,df_raw_detrended_log,df_raw_deseasonalized_log])
df_elasticity =final_df[['ID','name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]


In [ ]:
# for column in x_values_mod.columns[1:]:
#     column_points = []
#     for i in range(len(x_values_mod[column])):
#         if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]):
#             column_points.append((x_values_mod[column][i], y_values_mod[column][i]))
#     df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])
#     sns.regplot(x="x_value", y="y_value", data=df, fit_reg=True)
#     plt.xlabel('Price')
#     plt.ylabel('Quantity')
#     plt.show()

#     #Linear Regression Model
# #     x_value = df['x_value']
# #     y_value = df['y_value']
# #     X = sm.add_constant(x_value)
# #     model_raw = sm.OLS(y_value, X)
# #     result_raw = model_raw.fit()
#     x_value = [i for i in range(0, len(df))]
#     x_value = np.reshape(x_value, (len(x_value), 1))
#     y_value = df.values
#     model=LinearRegression()
#     model.fit(x_value,y_value)
#     # calculate trend
#     trend = model.predict(x_value)
#     # plot trend
#     plt.plot(y_value)
#     plt.plot(trend)
#     plt.show()
#     # detrend
#     detrended = [y_value[i]-trend[i] for i in range(0, len(df))]
#     # plot detrended
#     plt.plot(detrended)
#     plt.show()
#     print(detrended)
#     print(df)
    
#     #(Null Hypothesis test) Coefficient with a p value less than 0.05
# #     if result.f_pvalue < 0.05:
        
# #     rsquared = result.rsquared
# #     coefficient_pvalue = result.f_pvalue
# #     intercept, slope = result.params
# #     mean_price = np.mean(x_value)
# #     mean_quantity = np.mean(y_value)
# #     tintercept, t_score = result.tvalues

# #     #Price elasticity Formula
# #     price_elasticity = (slope)*(mean_price/mean_quantity)    

# #     #Append results into dictionary for dataframe
# #     results_values["name"].append(column)
# #     results_values["price_elasticity"].append(price_elasticity)
# #     results_values["price_mean"].append(mean_price)
# #     results_values["quantity_mean"].append(mean_quantity)
# #     results_values["intercept"].append(intercept)
# #     results_values['t_score'].append(t_score)
# #     results_values["slope"].append(slope)
# #     results_values["coefficient_pvalue"].append(coefficient_pvalue)
# #     results_values["rsquared"].append(rsquared*100)
        
# # final_df = pd.DataFrame.from_dict(results_values)
# # df_elasticity =final_df[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]


In [ ]:
final_df.sort_values(by=['ID','name'], ascending=True)

In [ ]:
good=final_df[final_df['rsquared']>=30]
good.sort_values(by='rsquared',ascending=False)

In [ ]:
final_df.to_csv("jay_test_excel_sku_R2.csv")

## Log Transformation

In [ ]:
df_LC=df_LC.drop('Sales',axis=1)
df_LC.head()
df_LC.head()

In [ ]:
df_LC_log=df_LC.copy()

In [ ]:
df_LC_log['LogPrice'] = np.log(df_LC_log['Price'])
df_LC_log['LogQty'] = np.log(df_LC_log['Qty'])

In [ ]:
df_LC_log.head()

In [ ]:
## Data Prep using pivot
x_pivot_log = df_LC_log.pivot(index= 'Year_Month' ,columns='LC' ,values='LogPrice')
x_values_log = pd.DataFrame(x_pivot_log.to_records())
print(x_values_log)

In [ ]:
percent_missing_x_log = x_values_log.isnull().sum() * 100 / len(x_values_log)
missing_value_df_x_log = pd.DataFrame({'column_name': x_values_log.columns,
                                 'percent_missing': percent_missing_x_log})
missing_value_df_x_log.sort_values(by=['percent_missing'])
mod_missing_value_df_x_log=missing_value_df_x_log[missing_value_df_x_log.percent_missing<=20]
mod_missing_value_df_x_log
mod_missing_value_df_x_log=mod_missing_value_df_x_log.reset_index(drop=True)
mod_missing_value_df_x_log['column_name']

In [ ]:
x_values_mod_log = x_values_log[x_values_log.columns[x_values_log.columns.isin(mod_missing_value_df_x_log['column_name'].astype('str'))]]
x_values_mod_log

In [ ]:
## Data Prep using pivot
y_pivot_log = df_LC_log.pivot(index= 'Year_Month' ,columns='LC' ,values='LogQty')
y_values_log = pd.DataFrame(y_pivot_log.to_records())
print(y_values_log)

In [ ]:
percent_missing_y_log = y_values_log.isnull().sum() * 100 / len(y_values_log)
missing_value_df_y_log = pd.DataFrame({'column_name': y_values_log.columns,
                                 'percent_missing': percent_missing_y_log})
# missing_value_df_y_log.sort_values(by=['percent_missing'])
mod_missing_value_df_y_log=missing_value_df_y_log[missing_value_df_y_log.percent_missing<=20]
# mod_missing_value_df_y_log
mod_missing_value_df_y_log=mod_missing_value_df_y_log.reset_index(drop=True)
mod_missing_value_df_y_log['column_name']

In [ ]:
y_values_mod_log = y_values_log[y_values_log.columns[y_values_log.columns.isin(mod_missing_value_df_y_log['column_name'].astype('str'))]]
y_values_mod_log

In [ ]:

points = []
results_values_log = {
    "name": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
}

In [ ]:
for column in x_values_mod_log.columns[1:]:
    column_points = []
    for i in range(len(x_values_mod_log[column])):
        if not np.isnan(x_values_mod_log[column][i]) and not np.isnan(y_values_mod_log[column][i]):
            column_points.append((x_values_mod_log[column][i], y_values_mod_log[column][i]))
    df_log = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])


    #Linear Regression Model
    x_value_log = df_log['x_value']
    y_value_log = df_log['y_value']
    X = sm.add_constant(x_value_log)
    model_log = sm.OLS(y_value_log, X)
    result_log = model_log.fit()
    
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result_log.rsquared
    coefficient_pvalue = result_log.f_pvalue
    intercept, slope = result_log.params
    mean_price = np.mean(x_value_log)
    mean_quantity = np.mean(y_value_log)
    tintercept, t_score = result_log.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log["name"].append(column)
    results_values_log["price_elasticity"].append(price_elasticity)
    results_values_log["price_mean"].append(mean_price)
    results_values_log["quantity_mean"].append(mean_quantity)
    results_values_log["intercept"].append(intercept)
    results_values_log['t_score'].append(t_score)
    results_values_log["slope"].append(slope)
    results_values_log["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log["rsquared"].append(rsquared*100)
        
final_df_log = pd.DataFrame.from_dict(results_values_log)
df_elasticity_log =final_df_log[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]


In [ ]:
final_df_log.sort_values(by=['rsquared'], ascending=False)

In [ ]:
good_log=final_df_log[final_df_log['rsquared']>=30]
good_log.sort_values(by='rsquared',ascending=False)

In [ ]:
final_df.to_csv("C:\\Users\\bhasraya\\OneDrive - DRiV Inc\\MP\\Adhoc\\Strut Assemblies - Tom Galla/excel_lc.csv")

## Instrumental Variables

In [ ]:
import matplotlib
from linearmodels import IV2SLS
from matplotlib.animation import FuncAnimation
from IPython import display

In [ ]:
df_IV=pd.read_csv('Strut_Assemblies_Python.csv')
df_IV.head()

In [ ]:
df_IV['Day'] = "1"
df_IV['Year']=df_IV['Year'].astype(str)
df_IV['Month_Num']=df_IV['Month_Num'].astype(str)
# df['Month1']=df['Month_Num']
# df['Year_Month']=pd.to_datetime(df[["Year", "Month", "Day"]])
df_IV['Year_Month']=pd.to_datetime(df_IV.Year+df_IV.Month_Num+df_IV.Day, format="%Y%m%d")
df_IV.rename(columns={'Price':'Price'}, inplace=True)
df_IV.rename(columns={'Units':'Qty'}, inplace=True)
df_IV.rename(columns={'Part/Sku':'Material'}, inplace=True)

In [ ]:
df_IV.isna().sum()

In [ ]:
df_IV=df_IV.dropna(axis=0, subset=('Price', ))

In [ ]:
#Check values in different Life Cycle
LC = df_IV['LC'].value_counts()
print("LC Percentage\n{}\n{}\n LC values count\n{}\n{}".format(
      50*"-", (LC / len(df_IV.index))[:5] * 100, 50*"-", LC[:5]))

## Data Exclusion

In [ ]:
#Aggregate the data
df_LC=df_IV[(df_IV.Year_Month<'2022-08-01') & (df_IV.Year_Month>='2020-07-01')]
# df_LC=df_IV.copy()
df_LC = df_LC.groupby(['Year_Month','LC']).agg({'Sales':'sum','Qty': 'sum', 'Income': 'mean' }).reset_index()
df_LC['Price']=df_LC['Sales']/df_LC['Qty']
df_LC.tail()

In [ ]:
df_LC['Price2'] = np.log(df_LC['Price'])
df_LC['Qty2'] = np.log(df_LC['Qty'])
df_LC['Income2'] = np.log(df_LC['Income'])

In [ ]:
# Mean, Max, Min by LCs
lcprices = df_LC.groupby('LC').agg({'Price' : ['mean', 'min', 'max']})
lcprices.columns = ['lc_price_mean','lc_price_min', 'lc_price_max']
lcprices["price_diff"] = lcprices["lc_price_max"] - lcprices["lc_price_min"]
lcprices = lcprices.reset_index()
lcprices.sort_values(by="price_diff", ascending = False, inplace = True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
     print(lcprices)

In [ ]:
#Box Plot
pct_threshold = 99.0
# censor below threshold
pct95facet = np.percentile(df_LC['Price'], pct_threshold)

box_dist = (
    ggplot(df_LC) + 
    geom_boxplot(
        aes(x = 'LC',
            y = 'Price'), colour="#1F3552", fill="#4271AE", 
                   alpha=0.7,
                   outlier_shape=".",
                   outlier_colour="red"       
    ) +
    labs(
        title ='Strut Assemblies by LC',
        x = 'LC',
        y = 'Price',
    ) +
    scale_y_continuous() +
    geom_hline(yintercept = pct95facet) + # add percentile solid line
    theme(axis_text_x = element_text(angle = 45, hjust = 1)) + 
    theme(figure_size=(20, 12)) + 
    theme(text=element_text(family="Tahoma", size=12)) +
    theme(axis_text_x=element_text(colour="black", size=12)) +
    theme(axis_text_y=element_text(colour="black", size=12))
)
box_dist

In [ ]:
## Data Prep using pivot
x_pivot = df_LC.pivot(index= 'Year_Month' ,columns='LC' ,values='Price')
x_values = pd.DataFrame(x_pivot.to_records())
print(x_values)

In [ ]:
percent_missing_x = x_values.isnull().sum() * 100 / len(x_values)
missing_value_df_x = pd.DataFrame({'column_name': x_values.columns,
                                 'percent_missing': percent_missing_x})
print(missing_value_df_x.sort_values(by=['percent_missing']))
mod_missing_value_df_x=missing_value_df_x[missing_value_df_x.percent_missing<=20]
mod_missing_value_df_x

In [ ]:
mod_missing_value_df_x=mod_missing_value_df_x.reset_index(drop=True)
x_values_mod = x_values[x_values.columns[x_values.columns.isin(mod_missing_value_df_x['column_name'].astype('str'))]]
x_values_mod

In [ ]:
y_pivot = df_LC.pivot( index = 'Year_Month',columns='LC', values='Qty')
y_values = pd.DataFrame(y_pivot.to_records())
print(y_values)
percent_missing_y = y_values.isnull().sum() * 100 / len(y_values)
missing_value_df_y = pd.DataFrame({'column_name': y_values.columns,
                                 'percent_missing': percent_missing_y})
print(missing_value_df_y.sort_values(by=['percent_missing']))
mod_missing_value_df_y=missing_value_df_y[missing_value_df_y.percent_missing<=20]
mod_missing_value_df_y=mod_missing_value_df_y.reset_index(drop=True)
y_values_mod = y_values[y_values.columns[y_values.columns.isin(mod_missing_value_df_y['column_name'].astype('str'))]]
y_values_mod

In [ ]:
z_pivot = df_LC.pivot( index = 'Year_Month',columns='LC', values='Income')
z_values = pd.DataFrame(z_pivot.to_records())
print(z_values)
percent_missing_z = z_values.isnull().sum() * 100 / len(z_values)
missing_value_df_z = pd.DataFrame({'column_name': z_values.columns,
                                 'percent_missing': percent_missing_z})
print(missing_value_df_z.sort_values(by=['percent_missing']))
mod_missing_value_df_z=missing_value_df_z[missing_value_df_z.percent_missing<=20]
mod_missing_value_df_z=mod_missing_value_df_z.reset_index(drop=True)
z_values_mod = z_values[z_values.columns[z_values.columns.isin(mod_missing_value_df_z['column_name'].astype('str'))]]
z_values_mod

In [ ]:
points = []
results_values = {
    "name": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
}

In [ ]:
for column in x_values_mod.columns[1:]:
    column_points = []
    for i in range(len(x_values_mod[column])):
        if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]) and not np.isnan(z_values_mod[column][i]):
            column_points.append((x_values_mod[column][i], y_values_mod[column][i], z_values_mod[column][i]))
    df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value','z_value'])
    sns.regplot(x="x_value", y="y_value", data=df, fit_reg=True)
    plt.xlabel('Price')
    plt.ylabel('Quantity')
    plt.show()
    sns.regplot(x="z_value", y="y_value", data=df, fit_reg=True)
    plt.xlabel('Income')
    plt.ylabel('Quantity')
    plt.show()

    #Linear Regression Model
    # x_value = df['x_value']
    # y_value = df['y_value']
    # X = sm.add_constant(x_value)
    # model = sm.OLS(y_value, X)
    # result = model.fit()
    
    model = IV2SLS.from_formula("y_value ~ 1 + [x_value ~ z_value]", df).fit()
    result = model.summary
    print(result)
    
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values["name"].append(column)
    results_values["price_elasticity"].append(price_elasticity)
    results_values["price_mean"].append(mean_price)
    results_values["quantity_mean"].append(mean_quantity)
    results_values["intercept"].append(intercept)
    results_values['t_score'].append(t_score)
    results_values["slope"].append(slope)
    results_values["coefficient_pvalue"].append(coefficient_pvalue)
    results_values["rsquared"].append(rsquared*100)
        
final_df = pd.DataFrame.from_dict(results_values)
df_elasticity =final_df[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]
